In [8]:
train_DIR = '/kaggle/input/photo-reconstruction/Dataset/Training_Data'
test_DIR = '/kaggle/input/photo-reconstruction/Dataset/Testing_Data'

In [9]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Compose
from PIL import Image

In [10]:
# BiLSTM Model
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# LOSS FUNCTION
class ReconstructionLoss(nn.Module):
    def __init__(self):
        super(ReconstructionLoss, self).__init__()

    def forward(self, output, target):
        return nn.MSELoss()(output, target)

In [11]:
# Image Dataset
class ImageDataset(Dataset):
    def __init__(self, masked_dir, unmasked_dir):
        self.masked_dir = masked_dir
        self.unmasked_dir = unmasked_dir
        self.transforms = Compose([ToTensor()])
        self.filenames = os.listdir(masked_dir)
        self.filenames.remove('masked_info.csv')
        self.masked_info = os.path.join(self.masked_dir, 'masked_info.csv')
    
    def __getitem__(self, idx):
        masked_path = os.path.join(self.masked_dir, self.filenames[idx])
        unmasked_path = os.path.join(self.unmasked_dir, self.filenames[idx])
        masked_image = Image.open(masked_path).convert('RGB')
        unmasked_image = Image.open(unmasked_path).convert('RGB')
        masked_tensor = self.transforms(masked_image)
        unmasked_tensor = self.transforms(unmasked_image)
        return masked_tensor, unmasked_tensor

    def __len__(self):
        return len(self.filenames)

In [18]:
import torch
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
# Define the model
model = BiLSTM(input_size=256, hidden_size=128, num_layers=2)

# Define the loss function
loss_fn = ReconstructionLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Load the data
cat_train_dataset = ImageDataset(masked_dir=f"{train_DIR}/Cat/Masked_Train", unmasked_dir=f"{train_DIR}/Cat/Unmasked_Train")
dog_train_dataset = ImageDataset(masked_dir=f"{train_DIR}/Dog/Masked_Train", unmasked_dir=f"{train_DIR}/Dog/Unmasked_Train")
elephant_train_dataset = ImageDataset(masked_dir=f"{train_DIR}/Elephant/Masked_Train", unmasked_dir=f"{train_DIR}/Elephant/Unmasked_Train")
tiger_train_dataset = ImageDataset(masked_dir=f"{train_DIR}/Tiger/Masked_Train", unmasked_dir=f"{train_DIR}/Tiger/Unmasked_Train")
train_dataset = ConcatDataset([cat_train_dataset, dog_train_dataset, elephant_train_dataset, tiger_train_dataset])
# Train Loader
train_loader = DataLoader(train_dataset, shuffle=True)

In [19]:
# model = BiLSTM(input_size=256, hidden_size=128, num_layers=8)

In [20]:
# Train the model
for epoch in range(10):
    training_loss = 0
    count = 0
    for i, (masked, unmasked) in enumerate(train_loader):
        # print(masked.shape)
        # print(masked[0])
        optimizer.zero_grad()
        output = model(masked[0])
        loss = loss_fn(output, unmasked)
        training_loss += loss.item()
        loss.backward()
        optimizer.step()
        count += 1
    print(f"Epoch {epoch}, MSE Loss = {training_loss}")
    torch.save(model,f'/kaggle/working/model{epoch}-8.pth')


Epoch 0, MSE Loss = 126.72352881450206
Epoch 1, MSE Loss = 82.95575771050062
Epoch 2, MSE Loss = 74.48835996165872
Epoch 3, MSE Loss = 69.7256657514954
Epoch 4, MSE Loss = 67.16253951238468
Epoch 5, MSE Loss = 65.16681404551491
Epoch 6, MSE Loss = 63.41975019604433
Epoch 7, MSE Loss = 62.26647049782332
Epoch 8, MSE Loss = 61.155868933186866
Epoch 9, MSE Loss = 59.98875722230878


In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
torch.save(model,f'/kaggle/working/bilstm.pth')

In [23]:
df = pd.read_csv(f'{test_DIR}/masked_info.csv')

In [25]:
import os
lst = os.listdir(test_DIR)
lst.remove('masked_info.csv')
data = []

# Test Each File in test_DIR file
for filename in lst:
  masked_path = os.path.join(test_DIR,filename)
#   Find All row col
  box1_row = df[df['filename'] == filename].iloc[0]['box1_row']
  box1_col = df[df['filename'] == filename].iloc[0]['box1_col']
  box2_row = df[df['filename'] == filename].iloc[0]['box2_row']
  box2_col = df[df['filename'] == filename].iloc[0]['box2_col']
  masked_image = Image.open(masked_path).convert('RGB')
  masked_tensor = Compose([ToTensor()])(masked_image)
  
  with torch.no_grad():
    output = model(masked_tensor)
  for i in range(75):
    for j in range(75):
      data.append([filename+'_box1_'+str(box1_row+i)+'_'+str(box1_col+j)+'_2',output[0][box1_row+i][box1_col+j].item()])
      data.append([filename+'_box1_'+str(box1_row+i)+'_'+str(box1_col+j)+'_1',output[1][box1_row+i][box1_col+j].item()])
      data.append([filename+'_box1_'+str(box1_row+i)+'_'+str(box1_col+j)+'_0',output[2][box1_row+i][box1_col+j].item()])
  for i in range(75):
    for j in range(75):
      data.append([filename+'_box2_'+str(box2_row+i)+'_'+str(box2_col+j)+'_2',output[0][box2_row+i][box2_col+j].item()])
      data.append([filename+'_box2_'+str(box2_row+i)+'_'+str(box2_col+j)+'_1',output[1][box2_row+i][box2_col+j].item()])
      data.append([filename+'_box2_'+str(box2_row+i)+'_'+str(box2_col+j)+'_0',output[2][box2_row+i][box2_col+j].item()])

final_df = pd.DataFrame(data, columns=['filename_box_pixel', 'Value'])
final_df.to_csv('submission.csv',index=False)

In [26]:
final_df

,filename_box_pixel,Value
0,Dog-Train (411).jpeg_box1_21_59_2,0.898908
1,Dog-Train (411).jpeg_box1_21_59_1,0.920529
2,Dog-Train (411).jpeg_box1_21_59_0,0.844931
3,Dog-Train (411).jpeg_box1_21_60_2,0.887779
4,Dog-Train (411).jpeg_box1_21_60_1,0.908767
...,...,...
6749995,Cat-Train (712).jpeg_box2_74_74_1,0.515302
6749996,Cat-Train (712).jpeg_box2_74_74_0,0.420458
6749997,Cat-Train (712).jpeg_box2_74_75_2,0.589004
6749998,Cat-Train (712).jpeg_box2_74_75_1,0.514908


In [ ]:
# Function to inpaint the masked images
def sample(filename,dir_path,output_image):
  box1_row = df[df['filename'] == filename].iloc[0]['box1_row']
  box1_col = df[df['filename'] == filename].iloc[0]['box1_col']
  box2_row = df[df['filename'] == filename].iloc[0]['box2_row']
  box2_col = df[df['filename'] == filename].iloc[0]['box2_col']
  imagepath = os.path.join(dir_path,filename)
  imagePIL = Image.open(imagepath).convert('RGB')
  image = Compose([ToTensor()])(imagePIL)
  output = image
  output[:][box1_row:box1_row+75][box1_col:box1_col+75] = output_image[:][box1_row:box1_row+75][box1_col:box1_col+75]
  output[:][box2_row:box2_row+75][box2_col:box2_col+75] = output_image[:][box2_row:box2_row+75][box2_col:box2_col+75]
  return output